<a href="https://colab.research.google.com/github/aurimas13/CodeAcademy-AI-Course/blob/main/Notebooks_In_Progress/Neural_Networks_for_Tabular_Data_6_L3_Demonstration_1_of_fastai_tabular_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tabular training

How to use the tabular application in fastai

To illustrate the tabular application, we will use the example of the [Adult dataset](https://archive.ics.uci.edu/ml/datasets/Adult) where we have to predict if a person is earning more or less than $50k per year using some general data.



In [ ]:
from fastai.tabular.all import *

We can download a sample of this dataset with the usual [untar_data](https://docs.fast.ai/data.external.html#untar_data) command:



In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/export.pkl')]

Then we can have a look at how the data is structured:


In [ ]:
df = pd.read_csv(path/'adult.csv')
# df.head()


Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in [TabularDataLoaders](https://docs.fast.ai/tabular.data.html#tabulardataloaders) factory methods:

The last part is the list of pre-processors we apply to our data:

- [Categorify](https://docs.fast.ai/tabular.core.html#categorify) is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
- [FillMissing](https://docs.fast.ai/tabular.core.html#fillmissing) will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
- [Normalize](https://docs.fast.ai/data.transforms.html#normalize) will normalize the continuous variables (subtract the mean and divide by the std)

To further expose what’s going on below the surface, let’s rewrite this utilizing fastai’s [TabularPandas](https://docs.fast.ai/tabular.core.html#tabularpandas) class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

Once we build our [TabularPandas](https://docs.fast.ai/tabular.core.html#tabularpandas) object, our data is completely preprocessed as seen below:



Now we can build our [DataLoaders](https://docs.fast.ai/data.core.html#dataloaders) again:



The [show_batch](https://docs.fast.ai/vision.gan.html#show_batch) method works like for every other application:

We can define a model using the [tabular_learner](https://docs.fast.ai/tabular.learner.html#tabular_learner) method. When we define our model, fastai will try to infer the loss function based on our `y_names` earlier.

**Note**: Sometimes with tabular data, your `y`’s may be encoded (such as 0 and 1). In such a case you should explicitly pass `y_block = CategoryBlock` in your constructor so `fastai` won’t presume you are doing regression.

And we can train that model with the `fit_one_cycle` method (the `fine_tune` method won’t be useful here since we don’t have a pretrained model)

We can then have a look at some predictions:



Or use the predict method on a row:



To get prediction on a new dataframe, you can use the `test_dl` method of the [DataLoaders](https://docs.fast.ai/data.core.html#dataloaders). That dataframe does not need to have the dependent variable in its column.

Then [Learner.get_preds](https://docs.fast.ai/learner.html#learner.get_preds) will give you the predictions:



# `fastai` with Other Libraries


As mentioned earlier, [TabularPandas](https://docs.fast.ai/tabular.core.html#tabularpandas) is a powerful and easy preprocessing tool for tabular data. Integration with libraries such as Random Forests and XGBoost requires only one extra step, that the `.dataloaders` call did for us. Let’s look at our `to` again. Its values are stored in a `DataFrame` like object, where we can extract the `cats`, `conts`, `xs` and `ys` if we want to:

Now that everything is encoded, you can then send this off to XGBoost or Random Forests by extracting the train and validation sets and their values:



And now we can directly send this in!